## Python Script to transform data

The script focuses on bring RECS data from 2001, 2005, 2009 and 2015 into 
1. common format
2. columns mapped correctly to the correct fields
3. calaculations done as required
4. elimination of columns not needed

### Rules:
1.	All columns starting with "Z" is dropped
2.	YearMade is converted to a range as per 2015 standards
3.	Drop 1997
4.	Club all electronics into 3 categories TVREL, PCOFFEQUIP, PHONE
5.	All comsumption will be BTU and KWH will not be used
6.	Calucaled Fields
    - "2001 - TOTROOMS - sum(BEDROOMS,NCOMBATH,NHAFBATH,OTHROOMS)"
	- TOTALBTU = TOTALBTUSPH + TOTALBTUWTH + TOTALBTUOTH
	- TOTALBTUSPH = BTULPSPH + BTUNGSPH + BTUFOSPH + BTUELSPH
	- TOTALBTUWTH = BTULPWTH + BTUNGWTH + BTUFOWTH + BTUELWTH
	- TOTALBTUOTH = BTULPAPL + BTUNGOTH + BTUFOAPL + BTUELOTH + BTUELFRG +BTUELCOL
	- TOTALDOLLAR = TOTALBTUSPH + TOTALBTUWTH + TOTALBTUOTH
	- TOTALDOLSPH = DOLLPSPH+ DOLNGSPH + DOLFOSPH + DOLELSPH
	- TOTALDOLWTH = DOLLPWTH+ DOLNGWTH+ DOLFOWTH+ DOLELWTH
	- TOTALDOLOTH = DOLLPOTH + DOLNGOTH+ DOLFOOTH + DOLELOTH + DOLELFRG + DOLELCOL


In [1]:
# import all dependecies
import pandas as pd
import os
import csv

from functools import reduce

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Global Notebook Variables and Initialization
dataFilePath = "dataforfinalproject/RawDataFiles"

codebook_path = "dataforfinalproject/RawDataFiles/Codebooks"

years = [2001, 2005, 2009, 2015]

dfs = {}

dataFiles = ['target_final_2001.csv', "RECS05alldata.csv", "recs2009_public.csv", "recs2015_public_v4.csv"]

col_list = ['2001_requiredCols.txt', '2005_requiredCols.txt', '2009_requiredCols.txt', '2015_requiredCols.txt']

final_colList = "final_columnList.txt"

In [3]:
# read the files into a dataframe and compress dataframe to have only required cols
for i, y in enumerate(years):
    print(y)
    df = pd.read_csv(os.path.join(dataFilePath, dataFiles[i]), low_memory=False)
    l = pd.read_csv(os.path.join(codebook_path, col_list[i]), header= None, names = ['cols']).cols.tolist()
    dfs[y] = df[l]

2001
2005
2009
2015


In [4]:
dfs[2005] = pd.read_csv(os.path.join(dataFilePath, dataFiles[1]), low_memory=False)

### Format and tranform 2001 data

In [5]:
# add Year of Survey to each of the datasets
for y in years:
    dfs[y]['RECSYEAR'] = y

In [6]:
# change columnname YearMade to YearMadeRange
dfs[2015].rename(columns = {"YEARMADERANGE":"YEARMADE"}, inplace = True)

# change columnname Basefin to Cellar
dfs[2005].rename(columns = {"BASEFIN":"CELLAR"}, inplace = True)
dfs[2015].rename(columns = {"BASEFIN":"CELLAR"}, inplace = True)
#dfs[2015].YEARMADE.head()

# change OCCUPYYRANGE to OCCUPYY
dfs[2015].rename(columns = {"OCCUPYYRANGE":"OCCUPYY"}, inplace = True)
dfs[2009].rename(columns = {"OCCUPYYRANGE":"OCCUPYY"}, inplace = True)

# change USENG to UGASHERE
dfs[2009].rename(columns = {"USENG":"UGASHERE"}, inplace = True)

# change TOAST to TOASTER in 2015
dfs[2015].rename(columns = {"TOAST":"TOASTER"}, inplace = True)

# change "STOVENFU" to "STOVENFUEL"
dfs[2001].rename(columns = {"STOVENFU":"STOVENFUEL"}, inplace = True)
dfs[2005].rename(columns = {"STOVENFU":"STOVENFUEL"}, inplace = True)

# rename columns OUTLGTNT(2001), NOUTLGTNT(2005, 2009) to LGTOUTNUM
dfs[2001].rename(columns = {"OUTLGTNT":"LGTOUTNUM"}, inplace = True)
dfs[2005].rename(columns = {"NOUTLGTNT":"LGTOUTNUM"}, inplace = True)
dfs[2009].rename(columns = {"NOUTLGTNT":"LGTOUTNUM"}, inplace = True)

# rename ORIGIN1 to Householder_Race in 2001 and 2005
dfs[2001].rename(columns = {"ORIGIN1":"HOUSEHOLDER_RACE"}, inplace = True)
dfs[2005].rename(columns = {"ORIGIN1":"HOUSEHOLDER_RACE"}, inplace = True)
dfs[2009].rename(columns = {"Householder_Race":"HOUSEHOLDER_RACE"}, inplace = True)

# rename CD65 and HD65 to CDD65 and HDD65
dfs[2005].rename(columns = {"CD65":"CDD65", "HD65":"HDD65"}, inplace = True)

# rename CORDS TO WOODAMT in 2001
dfs[2001].rename(columns = {"CORDS":"WOODAMT"}, inplace = True)

#change NGPAY to PGASHEAT for 2015
dfs[2015].rename(columns = {"NGPAY": "PGASHEAT"}, inplace = True)

In [7]:
# Calculate totrooms for 2001 data
dfs[2001]['TOTROOMS'] = dfs[2001].BEDROOMS + dfs[2001].NCOMBATH  + dfs[2001].NHAFBATH +dfs[2001].OTHROOMS


In [8]:
equipAux_type = ["REVERSE","WARMAIR","STEAMR","PERMELEC","PIPELESS","ROOMHEAT","WOODKILN","CARRYEL","CARRYKER",
                 "CHIMNEY","RANGE","DIFEQUIP"]
for e in equipAux_type:
    dfs[2015][e] = 0
    
dfs[2015].loc[dfs[2015].EQUIPAUXTYPE == 1,"CARRYEL"] = 1
dfs[2015].loc[dfs[2015].EQUIPAUXTYPE == 2,'WOODKILN'] = 1
dfs[2015].loc[dfs[2015].EQUIPAUXTYPE == 3,'PIPELESS'] = 1
dfs[2015].loc[dfs[2015].EQUIPAUXTYPE == 4,'CHIMNEY'] = 1
dfs[2015].loc[dfs[2015].EQUIPAUXTYPE == 9,'DIFEQUIP'] = 1

In [9]:
# For EquipAux types in 2001 to 2009, if value is -2 change it to 0
for e in equipAux_type:
    dfs[2001].loc[dfs[2001][e] == -2,e] = 0
    dfs[2005].loc[dfs[2005][e] == -2,e] = 0
    dfs[2009].loc[dfs[2009][e] == -2,e] = 0   


In [10]:
equipAux_fuel = ["ELECAUX","UGASAUX","LPGAUX","FOKRAUX","WOODAUX","OTHERAUX"]

for e in equipAux_fuel:
    dfs[2015][e] = 0
    dfs[2009][e] = 0
    

#2015 changes
dfs[2015].loc[dfs[2015].FUELAUX == 1,"UGASAUX"] = 1
dfs[2015].loc[dfs[2015].FUELAUX == 2,'LPGAUX'] = 1
dfs[2015].loc[dfs[2015].FUELAUX == 3,'FOKRAUX'] = 1
dfs[2015].loc[dfs[2015].FUELAUX == 5,'ELECAUX'] = 1
dfs[2015].loc[dfs[2015].FUELAUX == 7,'WOODAUX'] = 1
dfs[2015].loc[dfs[2015].FUELAUX == 21,'OTHERAUX'] = 1




In [11]:
# changes to 2001 to 2005
# combine the value for FO and Kero to one column and Solar into Other columns

dfs[2001]['FOKRAUX'] = 0
dfs[2005]['FOKRAUX'] = 0


dfs[2001].loc[(dfs[2001]['FOILAUX'] == 1) | (dfs[2001]['KEROAUX'] == 1),"FOKRAUX"] = 1
dfs[2005].loc[(dfs[2005]['FOILAUX'] == 1) | (dfs[2005]['KEROAUX'] == 1),"FOKRAUX"] = 1

dfs[2001].loc[(dfs[2001]['SOLARAUX'] == 1) | (dfs[2001]['OTHERAUX'] == 1),"OTHERAUX"] = 1
dfs[2005].loc[(dfs[2005]['SOLARAUX'] == 1) | (dfs[2005]['OTHERAUX'] == 1),"OTHERAUX"] = 1


In [12]:
for e in equipAux_fuel:
    dfs[2001].loc[dfs[2001][e] == -2,e] = 0
    dfs[2005].loc[dfs[2005][e] == -2,e] = 0   


In [13]:
# transform 2009 Auxillary fuel type to standard fuel types as specified in years (2001, 2005 and 2015)
equipAux_fuel_2009 = ["FURNFUEL","RADFUEL","PIPEFUEL","RMHTFUEL","HSFUEL","FPFUEL","RNGFUEL","DIFFUEL"]
for e in equipAux_fuel_2009:
    dfs[2009].loc[dfs[2009][e] == 1,"UGASAUX"] = 1
    dfs[2009].loc[dfs[2009][e] == 2,'LPGAUX'] = 1
    dfs[2009].loc[(dfs[2009][e] == 3) | (dfs[2009][e] == 4) ,'FOKRAUX'] = 1
    dfs[2009].loc[dfs[2009][e] == 5,'ELECAUX'] = 1
    dfs[2009].loc[dfs[2009][e] == 7,'WOODAUX'] = 1
    dfs[2009].loc[dfs[2009][e] > 7,'OTHERAUX'] = 1
    

In [14]:
# The year Column in 2009 is actual year instead of coded so convert to Year Category
yr_bins = [0, 1951, 1960,1970, 1980,1990, 2000, 2010, 2019]
yr_labels = [1,2,3,4,5,6,7,8]

dfs[2009].rename(columns = {"YEARMADE": "YRBUILT"}, inplace = True)
dfs[2009]['YEARMADE'] = pd.cut(dfs[2009]['YRBUILT'], labels = yr_labels, bins = yr_bins)
dfs[2009].YEARMADE.head()


0    7
1    6
2    3
3    5
4    5
Name: YEARMADE, dtype: category
Categories (8, int64): [1 < 2 < 3 < 4 < 5 < 6 < 7 < 8]

In [17]:
# Impute the value for Energy Star Dishwasher for 2001 based on other years
# currently go with -9 as no data has been recorded for this
dfs[2001]['ESDISHW'] = -9

In [18]:
# Combine all categories of BTU usage into BTUXXOTH
# --- Electric
dfs[2001]['BTUELOTH'] = dfs[2001][["BTUELAPL","BTUELFZZ","BTUELDWH","BTUELCDR"]].sum(axis = 1)
dfs[2005]['BTUELOTH'] = dfs[2005][["BTUELAPL","BTUELFZZ","BTUELDWH","BTUELCDR"]].sum(axis = 1)
dfs[2015]['BTUELOTH'] = dfs[2015][["BTUELRFG1","BTUELRFG2","BTUELFRZ","BTUELCOK","BTUELMICRO","BTUELCW","BTUELCDR",\
                                "BTUELDWH","BTUELLGT","BTUELTVREL","BTUELTV1","BTUELTV2","BTUELAHUHEAT","BTUELAHUCOL",\
                                   "BTUELEVAPCOL","BTUELCFAN","BTUELDHUM","BTUELHUM","BTUELPLPMP","BTUELHTBPMP","BTUELHTBHEAT",\
                                   "BTUELNEC"]].sum(axis = 1)

# --- LPG
dfs[2001].rename(columns = {"BTULPAPL":"BTULPOTH"}, inplace = True)
dfs[2005].rename(columns = {"BTULPAPL":"BTULPOTH"}, inplace = True)
dfs[2015]['BTULPOTH'] = dfs[2015][["BTULPCOK","BTULPCDR","BTULPNEC"]].sum(axis = 1)

# --- Natural Gas
dfs[2001].rename(columns = {"BTUNGAPL":"BTUNGOTH"}, inplace = True)
dfs[2005].rename(columns = {"BTUNGAPL":"BTUNGOTH"}, inplace = True)
dfs[2015]['BTUNGOTH'] = dfs[2015][["BTUNGCOK","BTUNGCDR","BTUNGPLHEAT","BTUNGHTBHEAT","BTUNGNEC"]].sum(axis = 1)

# --- Fuel Oil
dfs[2001]['BTUFOOTH'] = dfs[2001][["BTUFOAPL","BTUKRAPL"]].sum(axis = 1)
dfs[2005].rename(columns = {"BTUFOAPL":"BTUFOOTH"}, inplace = True)
dfs[2015].rename(columns = {"BTUFONEC":"BTUFOOTH"}, inplace = True)

# for 2001, merge the total for BTU from FO and Kerosene into one col
dfs[2001]['BTUFOSPH'] = dfs[2001][["BTUFOSPH","BTUKRSPH"]].sum(axis = 1)
dfs[2001]['BTUFOWTH'] = dfs[2001][["BTUFOWTH","BTUKRWTH"]].sum(axis = 1)


In [19]:
# Combine all categories of DOL usage into DOLXXOTH
# --- Electric
dfs[2001]['DOLELOTH'] = dfs[2001][["DOLELAPL","DOLELFZZ","DOLELDWH","DOLELCDR"]].sum(axis = 1)
dfs[2005]['DOLELOTH'] = dfs[2005][["DOLELAPL","DOLELFZZ","DOLELDWH","DOLELCDR"]].sum(axis = 1)
dfs[2015]['DOLELOTH'] = dfs[2015][["DOLELRFG1","DOLELRFG2","DOLELFRZ","DOLELCOK","DOLELMICRO","DOLELCW","DOLELCDR",\
                                   "DOLELDWH","DOLELLGT","DOLELTVREL","DOLELTV1","DOLELTV2","DOLELAHUHEAT","DOLELAHUCOL",\
                                   "DOLELCFAN","DOLELDHUM","DOLELHUM","DOLELPLPMP","DOLELHTBPMP","DOLELHTBHEAT",\
                                   "DOLELNEC"]].sum(axis = 1)

# --- LPG
dfs[2001].rename(columns = {"DOLLPAPL":"DOLLPOTH"}, inplace = True)
dfs[2005].rename(columns = {"DOLLPAPL":"DOLLPOTH"}, inplace = True)
dfs[2015]['DOLLPOTH'] = dfs[2015][["DOLLPCOK","DOLLPCDR","DOLLPNEC"]].sum(axis = 1)

# --- Natural Gas
dfs[2001].rename(columns = {"DOLNGAPL":"DOLNGOTH"}, inplace = True)
dfs[2005].rename(columns = {"DOLNGAPL":"DOLNGOTH"}, inplace = True)
dfs[2015]['DOLNGOTH'] = dfs[2015][["DOLNGCOK","DOLNGCDR","DOLNGPLHEAT","DOLNGHTBHEAT","DOLNGNEC"]].sum(axis = 1)

# --- Fuel Oil
dfs[2001]['DOLFOOTH'] = dfs[2001][["DOLFOAPL","DOLKRAPL"]].sum(axis = 1)
dfs[2005].rename(columns = {"DOLFOAPL":"DOLFOOTH"}, inplace = True)
dfs[2015].rename(columns = {"DOLFONEC":"DOLFOOTH"}, inplace = True)

# for 2001, merge the total for DOL from FO and Kerosene into one col
dfs[2001]['DOLFOSPH'] = dfs[2001][["DOLFOSPH","DOLKRSPH"]].sum(axis = 1)
dfs[2001]['DOLFOWTH'] = dfs[2001][["DOLFOWTH","DOLKRWTH"]].sum(axis = 1)



In [20]:
# Calculate the TOTAL BTU VALUES
# --- BY end use
# TOTALBTU = TOTALBTUSPH + TOTALBTUWTH + TOTALBTUOTH
# TOTALBTUSPH = BTULPSPH + BTUNGSPH + BTUFOSPH + BTUELSPH
# TOTALBTUWTH = BTULPWTH + BTUNGWTH + BTUFOWTH + BTUELWTH
# TOTALBTUOTH = BTULPAPL + BTUNGOTH + BTUFOAPL + BTUELOTH + BTUELFRG +BTUELCOL
# TOTALDOLLAR = TOTALDOLSPH + TOTALDOLWTH + TOTALDOLOTH
# TOTALDOLSPH = DOLLPSPH+ DOLNGSPH + DOLFOSPH + DOLELSPH
# TOTALDOLWTH = DOLLPWTH+ DOLNGWTH+ DOLFOWTH+ DOLELWTH
# TOTALDOLOTH = DOLLPOTH + DOLNGOTH+ DOLFOOTH + DOLELOTH + DOLELFRG + DOLELCOL

# --- Totals by end use
for y in years:
    dfs[y]['TOTALBTUSPH'] = dfs[y][["BTULPSPH","BTUNGSPH","BTUFOSPH","BTUELSPH"]].sum(axis = 1)
    dfs[y]['TOTALBTUWTH'] = dfs[y][["BTULPWTH","BTUNGWTH","BTUFOWTH","BTUELWTH"]].sum(axis = 1)
    dfs[y]['TOTALBTUOTH'] = dfs[y][["BTULPOTH","BTUNGOTH","BTUFOOTH","BTUELOTH","BTUELCOL","BTUELRFG"]].sum(axis = 1)
    dfs[y]['TOTALBTU'] = dfs[y][["TOTALBTUSPH","TOTALBTUWTH","TOTALBTUOTH"]].sum(axis = 1)
    dfs[y]['TOTALDOLSPH'] = dfs[y][["DOLLPSPH","DOLNGSPH","DOLFOSPH","DOLELSPH"]].sum(axis = 1)
    dfs[y]['TOTALDOLWTH'] = dfs[y][["DOLLPWTH","DOLNGWTH","DOLFOWTH","DOLELWTH"]].sum(axis = 1)
    dfs[y]['TOTALDOLOTH'] = dfs[y][["DOLLPOTH","DOLNGOTH","DOLFOOTH","DOLELOTH","DOLELCOL","DOLELRFG"]].sum(axis = 1)
    dfs[y]['TOTALDOLLAR'] = dfs[y][["TOTALDOLSPH","TOTALDOLWTH","TOTALDOLOTH"]].sum(axis = 1)
    
   

In [21]:
# Combine TV/AUDIO/STEREO into one col TVAUDIOEQUIP
# 2001 - "TVCOLOR", "BIGTV", "VCR","STEREO","BOOMBOX","COMPCTST","COMPNTST","OTHSTER"   

dfs[2001]['TVAUDIOEQUIP'] = dfs[2001][["TVCOLOR", "BIGTV", "VCR","STEREO","BOOMBOX","COMPCTST","COMPNTST","OTHSTER"]].\
                                    sum(axis = 1)

#2005 - first replace 99 with 0 and then add all fields
appl_2005 = ["TVCOLOR","BIGTV","PLASMANUM","VCR","DVD","NCOMBOVCRDVD","PLAYSTA","STEREO","BOOMBOX","COMPCTST","COMPNTST","OTHSTER"]
for a in appl_2005:
    dfs[2005][a] = dfs[2005][a].apply(lambda r : r if r != 99 else 0)

dfs[2005]['TVAUDIOEQUIP'] = dfs[2005][appl_2005].sum(axis=1)

#2009 - 
appl_2009 = ["TVCOLOR","COMBODVR1","DVR1","DIGITSTB1","PLAYSTA1","COMBOVCRDVD1","VCR1","DVD1","TVAUDIOSYS1","OTHERSTB1","CABLESAT2","COMBODVR2","DVR2","DIGITSTB2","PLAYSTA2","COMBOVCRDVD2","VCR2","DVD2","TVAUDIOSYS2","OTHERSTB2","CABLESAT3","COMBODVR3","DVR3","DIGITSTB3","PLAYSTA3","COMBOVCRDVD3","VCR3","DVD3","TVAUDIOSYS3","OTHERSTB3"]

for a in appl_2009:
    dfs[2009][a] = dfs[2009][a].apply(lambda r : r if r != -2 else 0)

dfs[2009]['TVAUDIOEQUIP'] = dfs[2009][appl_2009].sum(axis=1)

#2015
appl_2015 = ["TVCOLOR","CABLESAT","COMBODVR","SEPDVR","PLAYSTA","DVD","VCR","INTSTREAM","TVAUDIOSYS"]
for a in appl_2015:
    dfs[2015][a] = dfs[2015][a].apply(lambda r : r if r != -2 else 0)

dfs[2015]['TVAUDIOEQUIP'] = dfs[2015][appl_2015].sum(axis = 1)

In [22]:
# Combine PC and OFFICE  into one col PCOFFEQUIP

pc_2001 = ["NUMPC","LAPTOPPC","PCPRINT","FXCOPIER","FAX","COPIER"]
for p in pc_2001:
    dfs[2001][p] = dfs[2001][p].apply(lambda r : r if r != 99 else 0)
    dfs[2001][p] = dfs[2001][p].apply(lambda r : r if r != 9 else 0)
    
dfs[2001]['PCOFFEQUIP'] = dfs[2001][pc_2001].sum(axis = 1)

#2005 - first replace 99 or 9 with 0 and then add all fields
pc_2005 = ["NUMPC","LAPTOPPC","PCPRINT","FXCOPIER","FAX","COPIER"]
monitor = ["MONITOR1","MONITOR2","MONITOR3"]
for p in pc_2005:
    dfs[2005][p] = dfs[2005][p].apply(lambda r : r if (r != 99) else 0)
    dfs[2005][p] = dfs[2005][p].apply(lambda r : r if (r != 9) else 0)
for m in monitor:
    dfs[2005][m] = dfs[2005][m].apply(lambda r : r if (r != 99) else 0)
    dfs[2005][m] = dfs[2005][m].apply(lambda r : r if (r != 9) else 0)
    dfs[2005][m] = dfs[2005][m].apply(lambda r : r if (r != 2) else 1)    
    
dfs[2005]['PCOFFEQUIP'] = dfs[2005][pc_2005].sum(axis = 1) + dfs[2005][monitor].sum(axis = 1)

#2009 - 
pc_2009 = ["NUMPC","PCPRINT","FAX","COPIER"]
monitor = ["MONITOR1","MONITOR2","MONITOR3"]
for p in pc_2009:
    dfs[2009][p] = dfs[2009][p].apply(lambda r : r if (r != -2) else 0)
for m in monitor:
    dfs[2009][m] = dfs[2009][m].apply(lambda r : r if (r != 0) else 1)
    dfs[2009][m] = dfs[2009][m].apply(lambda r : r if (r != -2) else 0)    
    
dfs[2009]['PCOFFEQUIP'] = dfs[2009][pc_2009].sum(axis = 1) + dfs[2009][monitor].sum(axis = 1)

#2015
pc_2015 = ["DESKTOP","NUMLAPTOP","NUMTABLET","ELPERIPH"]

dfs[2015]['PCOFFEQUIP'] = dfs[2015][pc_2015].sum(axis = 1)

In [23]:
dfs[2015][["NUMSMPHONE","CELLPHONE"]].max()

NUMSMPHONE     8
CELLPHONE     10
dtype: int64

In [24]:
# Combine All phone and phone related   into one col PHONE

ph_01_05 = ["NOCORD","CELLPHON","ANSMACH"]
 
#2001
dfs[2001]['PHONE'] = dfs[2001][ph_01_05].sum(axis = 1)
#2005
dfs[2005]['PHONE'] = dfs[2005][ph_01_05].sum(axis = 1)

#2009 - 
ph_2009 = ["NOCORD","ANSMACH"]
dfs[2009]['PHONE'] = dfs[2009][ph_2009].sum(axis = 1)

#2015
ph_2015 = ["NUMSMPHONE","CELLPHONE"]

dfs[2015]['PHONE'] = dfs[2015][ph_2015].sum(axis = 1)

In [25]:
print(len(dfs[2001].columns))
print(len(dfs[2005].columns))
print(len(dfs[2009].columns))
print(len(dfs[2015].columns))

237
1090
248
277


In [26]:
fnl_cols = pd.read_csv(os.path.join(codebook_path, final_colList), header= None, names = ['cols']).cols.tolist()
len(fnl_cols)

199

In [27]:
print(f"columns with NAN Values in 2001 datase : {dfs[2001].columns[dfs[2001].isna().any()].tolist()}")
print(f"columns with NAN Values in 2005 datase : {dfs[2005].columns[dfs[2005].isna().any()].tolist()}")
print(f"columns with NAN Values in 2009 datase : {dfs[2009].columns[dfs[2009].isna().any()].tolist()}")
print(f"columns with NAN Values in 2015 datase : {dfs[2015].columns[dfs[2015].isna().any()].tolist()}")

columns with NAN Values in 2001 datase : []
columns with NAN Values in 2005 datase : ['ORIGELS', 'KAVALNG', 'ORIGNGQ', 'CUFEETNG', 'BTUNG', 'BTUNGOTH', 'BTUNGWTH', 'BTUNGSPH', 'DOLLARNG', 'DOLNGOTH', 'DOLNGWTH', 'DOLNGSPH']
columns with NAN Values in 2009 datase : []
columns with NAN Values in 2015 datase : []


In [28]:
# This exists for only for 2005
# for the columns with NA, fill it with Mean value of the column
dfs[2005].fillna(dfs[2005].mean(), inplace = True)

print(f"columns with NAN Values in 2005 dataset after NA FIX : {dfs[2005].columns[dfs[2005].isna().any()].tolist()}")

columns with NAN Values in 2005 dataset after NA FIX : ['ORIGELS', 'KAVALNG', 'ORIGNGQ', 'CUFEETNG', 'BTUNG', 'BTUNGOTH', 'BTUNGWTH', 'BTUNGSPH', 'DOLLARNG', 'DOLNGOTH', 'DOLNGWTH', 'DOLNGSPH']


In [29]:
dfs_frame = [dfs[2001][fnl_cols],dfs[2005][fnl_cols],dfs[2009][fnl_cols],dfs[2015][fnl_cols]]


In [34]:
dfs[2015][fnl_cols].shape

(5686, 199)

In [35]:
# combined_DF = pd.DataFrame(columns = fnl_cols)
# combined_DF = dfs_list[2].append(dfs_list[3], ignore_index = True)
combined_df = pd.concat(dfs_frame)

ValueError: Plan shapes are not aligned

In [134]:
print(f"columns with NAN Values in combined datase : {combined_df.columns[combined_df.isna().any()].tolist()}")

columns with NAN Values in combined datase : []


In [135]:
print(f"Total Count of records in Combined dataset: {combined_df.DOEID.count()}")

Total Count of records in Combined dataset: 26973


In [136]:
#write file to csv
combined_df.to_csv(os.path.join("./dataforfinalproject", "RECS_COMBINED_DATA.csv"), index = False)